In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

[[Home](/)]
[[Webapps](/jupyter/voila/tree/webapps)]

# Google searches



In [2]:
import ipywidgets as widgets
import pandas as pd
from IPython.display import HTML, display
from ipywidgets import Layout

#from ipywidgets import HTML, Layout
from markdown import markdown
from stratosphere import Stratosphere, options

In [3]:
pd.options.display.max_rows = 10
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 100

In [4]:
kb = Stratosphere(options.get("db.url_kb"))

In [5]:
def get_google_searches():
    q = """
        SELECT
            id,
            json_extract(data, '$.q') as query
        FROM entities
        WHERE type = "google.com/search_query"
    """
    df = kb.db.pandas(q)

    return df.set_index("query").to_dict(orient="index")


# get_google_searches()

In [6]:
def get_google_results(entity_id):
    q = f"""
    SELECT
        e.id,
        e.type,
        r.src,
        json_extract(e.data, '$.text') as text,
        json_extract(e.data, '$.url') as url
    FROM entities e
    LEFT JOIN relationships r ON e.id = r.dst
    WHERE
        e.type = "google.com/search_result" AND
        r.src = "{entity_id}"
    """
    df = kb.db.pandas(q)
    return df


# get_google_results('1d9b08d14cbfb686c2e1e760b017d384')

In [7]:
# Imports


options = get_google_searches()


if len(options) == 0:
    options = {"": ""}
    value = ""
else:
    value = list(options.values())[0]

w_searches = widgets.Select(
    options=options, value=value, rows=20, disabled=False, layout=Layout(width="99%", height="180px")
)

In [8]:
w_out = widgets.Output()

In [9]:
def render_img_url(url):
    return '<img src="' + url + '" style=max-height:124px;"/>'


def render_href(href):
    return f"<a href={href}>{href}</a>"


def w_searches_onchange(change):
    mks = []

    if w_searches.value == "":
        return

    if change["name"] == "value":
        w_out.clear_output()
        with w_out:
            df = get_google_results(w_searches.value["id"])

            for idx, row in df.iterrows():
                text = row.text if row.text is not None else row.url

                from urllib.parse import urlparse

                domain = urlparse(row.url).netloc

                mks.append(
                    f"{idx+1}. [[e](/jupyter/voila/render/webapps/entity-overview.ipynb?entity_id={row.id})]"
                    f" [{str(text)}]({row.url}) - **{domain}**"
                )

            cols = ["id", "text", "url"]
            df = df[cols]
            html = markdown("\n".join(mks))
            html = html.replace("<ol>", '<ol style="list-style-position: inside;">')
            html = html.replace("<a ", '<a target="_blank" ')

            display(HTML(html))


w_searches.observe(w_searches_onchange)

w_searches_onchange({"name": "value"})

In [10]:
display(w_searches)
display(w_out)

Select(layout=Layout(height='180px', width='99%'), options={'bellingcat tools': {'id': '9087537177b531b83175eb…

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<IPython.core.display.HTML object>', '…